

---
---
* Installing needed Libraries :

---





In [48]:
!pip install numpy pandas joblib matplotlib scikit-learn

In [49]:
!pip install numpy pandas joblib matplotlib scikit-learn tk

In [50]:
!pip install flask flask-ngrok joblib

In [51]:
!pip install pyngrok
!ngrok authtoken 2tJcCOcKawyNHUrD3fx8ZGX7mqf_UpyxfMvX1tmYtgH2vSdR

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml




---
---
* Importing Needed Libraries :



---




In [52]:
import pandas as pd
import numpy as np
import joblib
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy import stats
from google.colab import files
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import joblib
import datetime
from pyngrok import ngrok



---
* Loading the dataset :
---



In [53]:
# Load dataset
df=pd.read_csv('/content/first inten project.csv')



---
* Dataset Preprocessing :
  - Checking for Null values, DataTypes , Memory Usage  .  
  - Checking for whiteSpaces and remove them  .
---



In [54]:
# 1. Data Preprocessing
print("* Displaying the infos of the DataSet : ")
print(df.info())  # Check for null values and data types
print("#####################################################################################################################################")
print("* Checking for the existance of Null Values ")
print(df.isnull().sum())  # Check missing values
print("#####################################################################################################################################")
df.columns = df.columns.str.strip()  # Remove white spaces in column names

* Displaying the infos of the DataSet : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36285 entries, 0 to 36284
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Booking_ID                36285 non-null  object 
 1   number of adults          36285 non-null  int64  
 2   number of children        36285 non-null  int64  
 3   number of weekend nights  36285 non-null  int64  
 4   number of week nights     36285 non-null  int64  
 5   type of meal              36285 non-null  object 
 6   car parking space         36285 non-null  int64  
 7   room type                 36285 non-null  object 
 8   lead time                 36285 non-null  int64  
 9   market segment type       36285 non-null  object 
 10  repeated                  36285 non-null  int64  
 11  P-C                       36285 non-null  int64  
 12  P-not-C                   36285 non-null  int64  
 13  average price       



---
* Convert to date time format:
---



In [55]:
# Ensure 'date of reservation' is a string before using .str accessor
df['date of reservation'] = df['date of reservation'].astype(str)
# Convert to datetime, handling errors
df['date of reservation'] = pd.to_datetime(df['date of reservation'], errors='coerce')
# Check for any NaT values (invalid dates)
print("Rows with invalid dates:", df['date of reservation'].isna().sum())
#  drop NaT values
df.dropna(subset=['date of reservation'], inplace=True)  # Drop invalid dates


Rows with invalid dates: 37




---
* Checking for the existance of Outliers and handeling them :
---



In [56]:
# 2. Checking for the existence of outliers and Handle them
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns  # Select only numeric columns
Q1 = df[numeric_cols].quantile(0.25)  # First quartile
Q3 = df[numeric_cols].quantile(0.75)  # Third quartile
IQR = Q3 - Q1                         # Interquartile range
# Detecting outliers
outlier_condition = (df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR))
# Replacing outliers with NaN
df[numeric_cols] = df[numeric_cols].mask(outlier_condition, np.nan)
# Fill NaN values **only for numeric columns** using median
df[numeric_cols] = df[numeric_cols].apply(lambda col: col.fillna(col.median()))




---
* Feature engineering selection and extraction :
---



In [57]:
# 3. Feature Engineering
# Feature Selection (Dropping unnecessary features)
df.drop(columns=['Booking_ID', 'date of reservation'], inplace=True)  # Booking ID and date might not be useful



---
* Checking for Duplicates and remove them  :
---



In [58]:
# Check for duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")
# Remove duplicate rows
df = df.drop_duplicates()
print("Duplicates removed. New shape of dataset:", df.shape)

Number of duplicate rows: 11541
Duplicates removed. New shape of dataset: (24707, 15)




---
* Encoding for categorical Data :

---



In [59]:
# 4. Encoding Categorical Data
categorical_cols = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le



---
* Splitting the data into training , test and handling the imbalancing in classes :
---



In [60]:
# 5. Train-Test Split
X = df.drop(columns=['booking status'])  # Features
y = df['booking status']  # Target

# Train-Test Split (Stratified to maintain class distribution)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Apply SMOTE to the Training Set ONLY
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)



---
* Scalling and Fit transform for the data :
---



In [61]:
# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


---
* Models training and calculating accuracies and saving Models  :
---



In [62]:
if not os.path.exists("saved_models"):
    os.makedirs("saved_models")
# 6. Model Training and Accuracy Calculation
##  Linear Regression with Threshold-Based Classification
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)
y_pred_linear = linear_model.predict(X_test)
y_pred_linear = (y_pred_linear>= 0.5).astype(int)  # Apply threshold
linear_accuracy = accuracy_score(y_test, y_pred_linear)
print(f'Linear Regression Accuracy: {linear_accuracy:.4f}')
joblib.dump(linear_model, "saved_models/linear_regression.pkl")  # Save model
print(" Saved: Linear Regression Model")
print("#####################################################################################################################################")
##  Logistic Regression
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train,y_train)
y_pred_logistic = logistic_model.predict(X_test)
logistic_accuracy = accuracy_score(y_test, y_pred_logistic)
print(f'Logistic Regression Accuracy: {logistic_accuracy:.4f}')
joblib.dump(logistic_model, "saved_models/logistic_regression.pkl")  # Save model
print(" Saved: Logistic Regression Model")
print("#####################################################################################################################################")
## Polynomial Regression (Linear)
degrees = [2, 3, 4]
poly_Linear_accuracies = []
for d in degrees:
    poly = PolynomialFeatures(degree=d)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    poly_linear_model = LinearRegression()
    poly_linear_model.fit(X_train_poly, y_train)
    y_pred_poly_linear = (poly_linear_model.predict(X_test_poly) >= 0.5).astype(int)  # Apply threshold
    accuracy = accuracy_score(y_test, y_pred_poly_linear)
    poly_Linear_accuracies.append(accuracy)
    print(f'Polynomial Regression (Linear, Degree={d}) Accuracy: {accuracy:.4f}')
    joblib.dump(poly_linear_model, f"saved_models/poly_linear_d{d}.pkl")  # Save model
    print(f" Saved: Polynomial Linear Regression (Degree={d}) Model")
print("#####################################################################################################################################")
## Polynomial Regression (Logistic)
poly_Logistic_accuracies = []
for d in degrees:
    poly = PolynomialFeatures(degree=d)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    poly_model = LogisticRegression(max_iter=1000)
    poly_model.fit(X_train_poly, y_train)
    y_pred_poly = poly_model.predict(X_test_poly)
    poly_logistic_accuracy = accuracy_score(y_test, y_pred_poly)
    poly_Logistic_accuracies.append(poly_logistic_accuracy)
    print(f'Polynomial Regression (Logistic, Degree={d}) Accuracy: {poly_logistic_accuracy:.4f}')
    joblib.dump(poly_model, f"saved_models/poly_logistic_d{d}.pkl")  # Save model
    print(f" Saved: Polynomial Logistic Regression (Degree={d}) Model")

print("#####################################################################################################################################")
##  K-Nearest Neighbors (KNN)
neighbors = [3, 5, 7, 9, 11]
knn_accuracies = []
for k in neighbors:
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train, y_train)
    y_pred_knn = knn_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_knn)
    knn_accuracies.append(accuracy)
    print(f'KNN Accuracy (k={k}): {accuracy:.4f}')
    joblib.dump(knn_model, f"saved_models/knn_k{k}.pkl")  # Save model
    print(f" Saved: KNN (k={k}) Model")
print("#####################################################################################################################################")


Linear Regression Accuracy: 0.7379
 Saved: Linear Regression Model
#####################################################################################################################################
Logistic Regression Accuracy: 0.7391
 Saved: Logistic Regression Model
#####################################################################################################################################
Polynomial Regression (Linear, Degree=2) Accuracy: 0.7382
 Saved: Polynomial Linear Regression (Degree=2) Model
Polynomial Regression (Linear, Degree=3) Accuracy: 0.7614
 Saved: Polynomial Linear Regression (Degree=3) Model
Polynomial Regression (Linear, Degree=4) Accuracy: 0.7846
 Saved: Polynomial Linear Regression (Degree=4) Model
#####################################################################################################################################
Polynomial Regression (Logistic, Degree=2) Accuracy: 0.7364
 Saved: Polynomial Logistic Regression (Degree=2) Model
Polynomi

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Polynomial Regression (Logistic, Degree=4) Accuracy: 0.7843
 Saved: Polynomial Logistic Regression (Degree=4) Model
#####################################################################################################################################
KNN Accuracy (k=3): 0.7506
 Saved: KNN (k=3) Model
KNN Accuracy (k=5): 0.7580
 Saved: KNN (k=5) Model
KNN Accuracy (k=7): 0.7703
 Saved: KNN (k=7) Model
KNN Accuracy (k=9): 0.7723
 Saved: KNN (k=9) Model
KNN Accuracy (k=11): 0.7707
 Saved: KNN (k=11) Model
#####################################################################################################################################




---
* Models Downloading :
---



In [63]:
# Define the folder where models are saved
model_folder = "saved_models"
# Function to download a file if it exists
def download_file(filename):
    filepath = os.path.join(model_folder, filename)
    if os.path.exists(filepath):
        files.download(filepath)
        print(f"Downloaded: {filename}")
    else:
        print(f"File not found: {filename}")

#  Download Linear & Logistic Regression Models
download_file("linear_regression.pkl")
download_file("logistic_regression.pkl")

#  Download Polynomial Regression Models
for d in [2, 3, 4]:
    download_file(f"poly_linear_d{d}.pkl")
    download_file(f"poly_logistic_d{d}.pkl")

#  Download KNN Models
for k in [3, 5, 7, 9, 11]:
    download_file(f"knn_k{k}.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: linear_regression.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Downloaded: knn_k9.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: knn_k11.pkl




---
* Creating the web app using Flask :
---



In [74]:
%%writefile app.py
import os
import subprocess
import threading
from flask import Flask, render_template, request
import joblib
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import PolynomialFeatures
from pyngrok import ngrok

threshold_value = 0.5
app = Flask(__name__)

# Create the HTML file
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hotel Booking Prediction</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f4f4f4;
        }
        .container {
            max-width: 600px;
            margin: auto;
            background: #fff;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        h2 {
            text-align: center;
        }
        label {
            margin-top: 10px;
            display: block;
        }
        input[type="text"], input[type="number"], select {
            width: 100%;
            padding: 8px;
            margin: 5px 0;
            border: 1px solid #ccc;
            border-radius: 4px;
        }
        input[type="submit"] {
            background-color: #28a745;
            color: white;
            border: none;
            border-radius: 4px;
            padding: 10px;
            cursor: pointer;
        }
        input[type="submit"]:hover {
            background-color: #218838;
        }
        .result, .error {
            margin-top: 20px;
            padding: 10px;
            border-radius: 4px;
        }
        .result {
            background-color: #e7fce7;
            border: 1px solid #c3e6c3;
        }
        .error {
            background-color: #f8d7da;
            border: 1px solid #f5c6cb;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Hotel Booking Cancellation Prediction</h2>
        <form method="POST">
            <label for="adults">Number of Adults:</label>
            <input type="number" name="adults" required>

            <label for="children">Number of Children:</label>
            <input type="number" name="children" required>

            <label for="weekend_nights">Number of Weekend Nights:</label>
            <input type="number" name="weekend_nights" required>

            <label for="week_nights">Number of Week Nights:</label>
            <input type="number" name="week_nights" required>

            <label for="meal_type">Meal Type:</label>
            <select name="meal_type" required>
                <option value="Not Selected">Not Selected</option>
                <option value="Meal Plan 1">Meal Plan 1</option>
                <option value="Meal Plan 2">Meal Plan 2</option>
                <option value="Meal Plan 3">Meal Plan 3</option>
            </select>

            <label for="parking">Parking:</label>
            <input type="number" name="parking" required>

            <label for="room_type">Room Type:</label>
            <select name="room_type" required>
                <option value="Room_Type 1">Room Type 1</option>
                <option value="Room_Type 2">Room Type 2</option>
                <option value="Room_Type 3">Room Type 3</option>
                <option value="Room_Type 4">Room Type 4</option>
                <option value="Room_Type 5">Room Type 5</option>
                <option value="Room_Type 6">Room Type 6</option>
                <option value="Room_Type 7">Room Type 7</option>
            </select>

            <label for="lead_time">Lead Time:</label>
            <input type="number" name="lead_time" required>

            <label for="market_segment">Market Segment:</label>
            <select name="market_segment" required>
                <option value="Online">Online</option>
                <option value="Offline">Offline</option>
                <option value="Corporate">Corporate</option>
                <option value="Affiliation">Affiliation</option>
                <option value="Complementary">Complementary</option>
            </select>

            <label for="repeated">Repeated Guest (1 for Yes, 0 for No):</label>
            <input type="number" name="repeated" min="0" max="1" required>

            <label for="p_c">Probability of Cancellation:</label>
            <input type="text" name="p_c" required>

            <label for="p_not_c">Probability of Not Cancellation:</label>
            <input type="text" name="p_not_c" required>

            <label for="price">Price:</label>
            <input type="text" name="price" required>

            <label for="special_requests">Special Requests:</label>
            <input type="number" name="special_requests" required>

            <input type="submit" value="Predict">
        </form>

        {% if result %}
        <div class="result">
            <h3>Prediction Results:</h3>
            <p><strong>Timestamp:</strong> {{ result['Timestamp'] }}</p>
            <p><strong>User Inputs:</strong></p>
            <ul>
                {% for key, value in result['Input'].items() %}
                    <li><strong>{{ key }}:</strong> {{ value }}</li>
                {% endfor %}
            </ul>
            <p><strong>Booking Cancellation Prediction:</strong>
                {% if result['Cancellation Prediction'] == 'Likely to Cancel' %}
                    Likely to Cancel
                {% else %}
                    Not Likely to Cancel
                {% endif %}
            </p>
        </div>
        {% endif %}
        {% if error %}
        <div class="error">
            <strong>Error:</strong> {{ error }}
        </div>
        {% endif %}
    </div>
</body>
</html>
"""

# Write the HTML content to the index.html file
with open("templates/index.html", "w") as html_file:
    html_file.write(html_content)

# Function to start ngrok tunnel
def start_ngrok():
    public_url = ngrok.connect(5001)
    print(f" * Ngrok Tunnel: {public_url}")

ngrok_thread = threading.Thread(target=start_ngrok, daemon=True)
ngrok_thread.start()

# Load trained models (ensure the model paths are correct)
model_folder = "saved_models"
linear_model = joblib.load(os.path.join(model_folder, "linear_regression.pkl"))
logistic_model = joblib.load(os.path.join(model_folder, "logistic_regression.pkl"))
poly_models = {d: joblib.load(os.path.join(model_folder, f"poly_linear_d{d}.pkl")) for d in [2, 3, 4]}
poly_logistic_models = {d: joblib.load(os.path.join(model_folder, f"poly_logistic_d{d}.pkl")) for d in [2, 3, 4]}
knn_models = {k: joblib.load(os.path.join(model_folder, f"knn_k{k}.pkl")) for k in [3, 5, 7, 9, 11]}

# Mapping categorical inputs
meal_mapping = {"Not Selected": 0, "Meal Plan 1": 1, "Meal Plan 2": 2, "Meal Plan 3": 3}
room_mapping = {f"Room_Type {i}": i - 1 for i in range(1, 8)}
market_mapping = {"Online": 0, "Offline": 1, "Corporate": 2, "Affiliation": 3, "Complementary": 4}

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        try:
            # Collect input from form
            user_input = {
                "adults": int(request.form["adults"]),
                "children": int(request.form["children"]),
                "weekend_nights": int(request.form["weekend_nights"]),
                "week_nights": int(request.form["week_nights"]),
                "meal_type": meal_mapping[request.form["meal_type"]],
                "parking": int(request.form["parking"]),
                "room_type": room_mapping[request.form["room_type"]],
                "lead_time": int(request.form["lead_time"]),
                "market_segment": market_mapping[request.form["market_segment"]],
                "repeated": int(request.form["repeated"]),
                "p_c": float(request.form["p_c"]),
                "p_not_c": float(request.form["p_not_c"]),
                "price": float(request.form["price"]),
                "special_requests": int(request.form["special_requests"])
            }

            # Convert user input to numpy array for prediction
            input_array = np.array(list(user_input.values())).reshape(1, -1)

            # Make predictions
            linear_pred = linear_model.predict(input_array)[0]
            logistic_pred = logistic_model.predict(input_array)[0]
            poly_preds = {d: poly_models[d].predict(PolynomialFeatures(degree=d).fit_transform(input_array))[0] for d in [2, 3, 4]}
            poly_logistic_preds = {d: poly_logistic_models[d].predict(PolynomialFeatures(degree=d).fit_transform(input_array))[0] for d in [2, 3, 4]}
            knn_preds = {k: knn_models[k].predict(input_array)[0] for k in [3, 5, 7, 9, 11]}

            # Combine all predictions into a list
            all_predictions = [linear_pred, logistic_pred] + list(poly_preds.values()) + list(poly_logistic_preds.values()) + list(knn_preds.values())

            # Find the largest prediction value
            largest_prediction = max(all_predictions)

            # Determine cancellation prediction based on the largest value
            cancellation_prediction = 'Likely to Cancel' if largest_prediction > threshold_value else 'Not Likely to Cancel'

            # Save results to CSV
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            results = {
                "Timestamp": timestamp,
                "Input": user_input,
                "Cancellation Prediction": cancellation_prediction
            }
            df = pd.DataFrame([results])
            df.to_csv("predictions_log.csv", mode="a", header=False, index=False)

            # Return results with input
            return render_template("index.html", result=results)

        except Exception as e:
            return render_template("index.html", error=str(e))

    return render_template("index.html", result=None)

if __name__ == "__main__":
    app.run(port=5001)

Overwriting app.py


In [65]:
os.makedirs("templates", exist_ok=True)



---
* running the app :
---



In [ ]:
!python app.py

 * Serving Flask app 'app'
 * Debug mode: off
 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Ngrok Tunnel: NgrokTunnel: "https://bed6-34-80-222-204.ngrok-free.app" -> "http://localhost:5001"
127.0.0.1 - - [20/Feb/2025 20:09:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Feb/2025 20:09:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Feb/2025 20:18:41] "GET / HTTP/1.1" 200 -
